In [1]:
import wikipedia
import wikipediaapi 
import datetime
import pandas as pd
import numpy as np
from sklearn import feature_extraction
from sklearn import model_selection
import nltk
from nltk import tokenize
from nltk import word_tokenize, pos_tag
import string
from keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import multiclass
from sklearn import metrics
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
classes = ['sport', 'economy', 'politics', 'health', 'tourism', 
           'entertainment', 'culture', 'music', 'education', 
           'real estate', 'nutrition', 'science', 'history', 'technology'
          ]

In [3]:
def generate_data(classes, n):
    data = [];
    for i in classes:
#         topics = wikipedia.search(i, results = n)
#         for j in topics:
#         print(classes)
#         print("Class: ", i)
        for j in list(wikipedia.search(i,results = n)):
            wiki_wiki = wikipediaapi.Wikipedia('en')
            page_py = wiki_wiki.page(j)
            if(page_py.exists()):
#                 print("Class: ", i)
#                 print("Title: ", j)
#                     print("Section: ", page_py.sections[0].title)
#                     print("Text: ", page_py.text)
                for k in page_py.categories:
                    if i in k.lower(): 
#                         print("Class: ", i)
                        data.append((i, page_py.summary))
                        break
            else:
                continue
    return data

In [5]:
data = generate_data(classes, 10)

In [6]:
for i in classes:
    c = 0
    for j in range(0, len(data)):
        if(data[j][0] == i):
            c = c + 1
    print("%s %d" %(i, c))

sport 10
economy 4
politics 6
health 7
tourism 9
entertainment 5
culture 9
music 7
education 9
real estate 9
nutrition 8
science 8
history 6
technology 8
